In [1]:
import json
with open("config.json", "rt") as f:
    config_obj = json.load(f)
print(config_obj)
globals().update(config_obj)

{'dataset_path': '<path to your chb-mit directory> e.g ', 'clean_data_path': '<path to store clean & aligned dataset> e.g ./data_clean', 'label_output_path': '<path to store generated TOML files> e.g ./ref_labels', 'ignore_lst': []}


In [ ]:
get_id_func = lambda fn: int(fn[fn.rfind("_")+1:fn.rfind("h")])

In [ ]:
import scipy.io as scio
import numpy as np
import os

output_dir= clean_data_path
directory = dataset_path

with os.scandir(directory) as entries:
    for entry in entries:
        if entry.is_dir():
            subdir_path = entry.path

            # 在当前目录下创建同名子目录（如果不存在）
            target_dir = os.path.join(output_dir, entry.name)
            os.makedirs(target_dir, exist_ok=True)

            with os.scandir(subdir_path) as mats:
                data_header_lst = []
                
                mat_lst = []; shape_lst = []
                for fmat in mats:
                    if fmat.is_file and fmat.name.endswith(".mat"):
                        if "info" in fmat.name.lower():
                            info = scio.loadmat(fmat.path)
                            info["__header__"] = info["__header__"].decode()
                            info["fs"] = info["fs"].item()
                            info["seizure_begin"] = info["seizure_begin"].squeeze().tolist()
                            info["seizure_end"] = info["seizure_end"].squeeze().tolist()
                            info_path = os.path.join(target_dir, fmat.name[:-3]+"json")
                        else: 
                            mat_lst.append(fmat)
                            temp = scio.loadmat(fmat.path) # 第一遍读取：这里原理上只用读取文件头就可以，但是python生态没有提供方便读取老版本MAT文件头的API
                            shape_lst.append(temp["EEG"].shape)

                mat_lst.sort(key=lambda e: get_id_func(e.name))
                sum_shape = (shape_lst[0][0], sum(s[1] for s in shape_lst))
                result = np.memmap(os.path.join(target_dir, f'{entry.name}_eeg.npy'),  dtype='float32', mode='w+', shape=sum_shape)
                cur_idx = 0
                for fmat in mat_lst:
                        print(f"Reading {fmat.path}")
                        data = scio.loadmat(fmat.path) # 第二遍读取：用于实际数据处理
                        result[:, cur_idx:cur_idx+data["EEG"].shape[1]] = data["EEG"][:]
                        result.flush(); cur_idx += data["EEG"].shape[1]
                        data["__header__"] = data["__header__"].decode()
                        data["__shape__"] = data["EEG"].shape
                        del data["EEG"]
                        data_header_lst.append((fmat.name, data))
                        
                # data_header_lst.sort(key=lambda t: get_id_func(t[0]))
                with open(os.path.join(target_dir, entry.name+"_data.json"), "wt") as f:
                    json.dump(data_header_lst, f, indent=4)
                
                info["file"] = f'{entry.name}_eeg.npy' # result.filename; 
                info["shape"] = sum_shape
                with open(info_path, "wt") as f:
                    json.dump(info, f, indent=4)
            # print(f"Created directory: {target_dir}")

[en_US] TODO

[zh_CN]（可选步骤）测试上面生成的清洗数据集的正确性 

In [34]:
import datetime
import os
import json
output_dir=clean_data_path

meta_info = {}
with os.scandir(output_dir) as pats:
    for pat in pats:
        if pat.is_dir():
            meta_info[pat.name] = pat_meta = {}
            with os.scandir(pat.path) as jsons:
                # assert len(jsons) == 2
                sum_td = 0
                for fjson in jsons:
                    if fjson.name == (pat.name+"_eeg.dat"):
                        lf = np.memmap(fjson.path, dtype=np.float32, mode="r")
                        continue

                    with open(fjson.path, "rt") as f:
                        obj = json.load(f)
                        if fjson.name == (pat.name+"_info.json"):
                            pat_meta["fs"] = obj["fs"]
                            assert len(obj["seizure_begin"]) == len(obj["seizure_end"])
                            assert all([obj["seizure_begin"][k] > obj["seizure_end"][k-1] for k in range(1, len(obj["seizure_begin"]))])
                            assert all(map(lambda t: t[0] < t[1], zip(obj["seizure_begin"], obj["seizure_end"])))
                            pat_meta["beg"] = obj["seizure_begin"]; pat_meta["end"] = obj["seizure_end"]
                            pat_meta["shape"] = obj["shape"]
                        elif fjson.name == (pat.name+"_data.json"): 
                            assert len(obj) == get_id_func(obj[-1][0])
                            common_shape = obj[0][1]["__shape__"]
                            for idx, header in zip(range(1, len(obj)+1), obj):
                                assert get_id_func(header[0]) == idx
                                if idx != len(obj):
                                    assert header[1]["__shape__"] == common_shape
                                else:
                                    assert header[1]["__shape__"][0] == common_shape[0]
                                sum_td += header[1]["__shape__"][1]
                            first_mat_obj = scio.loadmat(os.path.join(directory, pat.name, obj[0][0])) 
                            last_mat_obj = scio.loadmat(os.path.join(directory, pat.name, obj[-1][0]))
                            pat_meta["nch"] = common_shape[0]
                        else:
                            raise AssertionError(f"Unknown Json File {fjson.path}")
                pat_meta["len"] = sum_td / pat_meta["fs"]
                assert pat_meta["beg"][0] >= 0 and pat_meta["end"][-1] <= pat_meta["len"]
                assert lf.size == pat_meta['nch'] * sum_td

                lf = lf.reshape(pat_meta["shape"])
                assert np.all(np.equal(lf[:, :first_mat_obj["EEG"].shape[1]], first_mat_obj["EEG"]))
                assert np.all(np.equal(lf[:, -(last_mat_obj["EEG"].shape[1]):], last_mat_obj["EEG"]))
                del lf, first_mat_obj, last_mat_obj

                print(f"{pat.name}: {datetime.timedelta(seconds=pat_meta['len'])}, {pat_meta['nch']} channels, {len(pat_meta['beg'])} onset(s)")

ID07: 2 days, 20:57:06, 75 channels, 4 onset(s)
ID03: 6 days, 14:23:01, 64 channels, 4 onset(s)
ID01: 12 days, 5:21:33, 88 channels, 2 onset(s)
ID04: 1 day, 16:41:37, 32 channels, 14 onset(s)
ID14: 6 days, 17:23:32, 24 channels, 60 onset(s)
ID18: 8 days, 13:04:22, 42 channels, 5 onset(s)
ID09: 1 day, 16:52:35, 48 channels, 27 onset(s)
ID06: 6 days, 1:59:12, 32 channels, 8 onset(s)
ID08: 5 days, 23:47:14, 61 channels, 70 onset(s)
ID15: 8 days, 3:55:00, 98 channels, 2 onset(s)
ID16: 7 days, 9:03:34, 34 channels, 5 onset(s)
ID10: 1 day, 18:23:24, 32 channels, 17 onset(s)
ID12: 7 days, 23:25:00, 56 channels, 9 onset(s)
ID11: 8 days, 20:13:02, 32 channels, 2 onset(s)
ID17: 5 days, 9:35:57, 60 channels, 2 onset(s)
ID02: 9 days, 19:13:05, 66 channels, 2 onset(s)
ID13: 4 days, 7:59:26, 64 channels, 7 onset(s)
ID05: 4 days, 13:24:22, 128 channels, 4 onset(s)
